In [2]:
import pandas as pd
#Misc_Val -> valore della feature
#Misc_Feature -> togliamo ??
#Condition 2 -> 2900 Norm su 2931  -> rimuovere
#Roof_Matl -> rimovere troppo uguali
#Piscina con campo a None
#Street -> 13 diversi da Pavel levare
#Utilities -> levare troppi pochi campi diversi
#anno ristrutturazione come unico Anno


In [4]:
x_train = pd.read_csv('x_train_cleaned.csv', index_col=0)
y_train = pd.read_csv('y_train_cleaned.csv', index_col=0)
x_test = pd.read_csv('x_test_cleaned.csv', index_col=0)

x_train.head(10)

,Lot_Frontage,Lot_Area,Overall_Qual,Overall_Cond,Year_Built,Year_Remod_Add,Mas_Vnr_Area,Exter_Qual,Exter_Cond,Bsmt_Qual,...,Sale_Type_b'New',Sale_Type_b'Oth',Sale_Type_b'VWD',Sale_Type_b'WD ',Sale_Condition_b'Abnorml',Sale_Condition_b'AdjLand',Sale_Condition_b'Alloca',Sale_Condition_b'Family',Sale_Condition_b'Normal',Sale_Condition_b'Partial'
0,141.0,31770.0,5,4,1960.0,1960.0,112.0,1,2,3,...,0,0,0,1,0,0,0,0,1,0
1,80.0,11622.0,4,5,1961.0,1961.0,0.0,1,2,3,...,0,0,0,1,0,0,0,0,1,0
2,81.0,14267.0,5,5,1958.0,1958.0,108.0,1,2,3,...,0,0,0,1,0,0,0,0,1,0
3,93.0,11160.0,6,4,1968.0,1968.0,0.0,2,2,3,...,0,0,0,1,0,0,0,0,1,0
4,74.0,13830.0,4,4,1997.0,1998.0,0.0,1,2,4,...,0,0,0,1,0,0,0,0,1,0
5,78.0,9978.0,5,5,1998.0,1998.0,20.0,1,2,3,...,0,0,0,1,0,0,0,0,1,0
6,41.0,4920.0,7,4,2001.0,2001.0,0.0,2,2,4,...,0,0,0,1,0,0,0,0,1,0
7,43.0,5005.0,7,4,1992.0,1992.0,0.0,2,2,4,...,0,0,0,1,0,0,0,0,1,0
8,39.0,5389.0,7,4,1995.0,1996.0,0.0,2,2,4,...,0,0,0,1,0,0,0,0,1,0
9,60.0,7500.0,6,4,1999.0,1999.0,0.0,1,2,3,...,0,0,0,1,0,0,0,0,1,0
